In [9]:
from csv import reader
from math import sqrt
from json import dumps


def sim(u, users):  # метрика сходства
    sim_dict = {}
    for v in range(len(users)):
        if u != users[v]:
            num = 0
            denom1 = 0
            denom2 = 0
            for i in range(len(u)):
                if u[i] != -1 and users[v][i] != -1:
                    num += u[i] * users[v][i]
                    denom1 += u[i] ** 2
                    denom2 += users[v][i] ** 2
            sim_dict[v] = round(num / (sqrt(denom1) * sqrt(denom2)), 3)
    sims = sorted(sim_dict.items(), key=lambda x: x[1], reverse=True)  # выводим результат в виде tuple (ключ, значение), упордоченного по убыванию значения
    return sims


def avg_rnk(u):  # средняя оценка пользователя
    avg = 0
    cnt = 0
    for i in range(len(u)):
        if u[i] != -1:
            avg += u[i]
            cnt += 1
    return round(avg / cnt, 3)

In [10]:
def r_ui(u, data, i, sims, knn):  # рассчет оценки фильма i для пользователя u
    num = 0
    denom = 0
    cnt = 0
    r_u = avg_rnk(u)
    for v in sims:
        if cnt != knn and data[v[0]][i] != -1:
            r_v = avg_rnk(data[v[0]])
            num += v[1] * (data[v[0]][i] - r_v)
            denom += abs(v[1])
        elif cnt == knn:
            break
        cnt += 1
    return round(r_u + num / denom, 3)

In [11]:
def compute_rnk(usr_id, data, context_place, context_day, knn):  # вычисление всех недостающий оценок пользоватея u, создание рекомендаций
    new_rnk = {}
    recommend = {}
    u = data[int(usr_id) - 1]
    sims = sim(u, data)
    for i in range(len(u)):
        if u[i] == -1:
            new_rnk["Movie " + str(i + 1)] = r_ui(u, data, i, sims, knn)
    for v in sims:
        if recommend:
            break
        for i in range(len(data[v[0]])):
                """
                Рекомендуем пользователю u фильм i, который смотрел самый схожый (по метрике)
                с ним пользователь дома в выходной день и который поставил этому фильму оценку
                выше, чем его средняя оценка просмотренных фильмов. Также выводим оценку,которую бы поставил
                пользователь u этому фильму.
                """
                if u[i] == -1 and data[v[0]][i] != -1 and (
                        context_day[v[0]][i] == " Sat" or context_day[v[0]][i] == " Sun") \
                        and context_place[v[0]][i] == " h" and data[v[0]][i] > avg_rnk(data[v[0]]):
                    recommend["Movie " + str(i + 1)] = new_rnk["Movie " + str(i + 1)]
                    break
   
    output = {
        "user": "User " + usr_id,
        1: new_rnk,
        2: recommend
    }
    print(output)

    file_name = "user" + str(usr_id) + ".json"
    json_dump = open(file_name, 'w')
    json_dump.write(dumps(output))
    json_dump.close()

In [12]:
def read_file(path):  # читаем содержимое файла и записывам его в лист
    with open(path, "r") as f_obj:
        rdr = reader(f_obj)
        next(rdr)
        return [[item for item in row[1:]] for row in rdr]

In [13]:
data = []
for i in read_file("data.csv"):
    data.append(list(map(int, i)))
place = read_file("context_place.csv")
day = read_file("context_day.csv")

usr = 0

while int(usr) < 1 or int(usr) > 40:
    usr = input("Enter user id: ")
    if int(usr) < 1 or int(usr) > 40:
        print("Incorrect id!")

compute_rnk(usr, data, place, day, 4)

Enter user id: 1
{'user': 'User 1', 1: {'Movie 1': 3.967, 'Movie 5': 2.712, 'Movie 6': 3.093, 'Movie 7': 3.714, 'Movie 10': 2.227, 'Movie 17': 1.458, 'Movie 23': 2.712, 'Movie 24': 2.974, 'Movie 27': 3.095}, 2: {'Movie 5': 2.712}}
